In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 安裝pytorch 人臉辨識facenet套件

In [2]:
!pip install facenet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.7 MB/s 


## import 用到的套件

In [3]:
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
import sys
sys.path.append(folder+"模組/")
import webcam2

In [4]:
import numpy as np
import pandas as pd
import time
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
from facenet_pytorch import MTCNN, InceptionResnetV1
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

## 初始化人臉檢測網路

In [5]:
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
resnet = InceptionResnetV1(pretrained='vggface2').eval() 

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [6]:
dataset = datasets.ImageFolder(folder+"資料集/photos") # 讀取照片資料夾 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # 資料夾名稱轉人名連結
def collate_fn(x):
    return x[0]
loader = torch.utils.data.DataLoader(dataset, collate_fn=collate_fn)
name_list = [] #空的姓名列表
embedding_list = [] #空的特徵向量

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True)
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

#儲存訓練資料
data = [embedding_list, name_list] 
torch.save(data, folder+'data.pt') # saving data.pt file

## 是否有GPU可以使用

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [8]:
load_data = torch.load(folder+'data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

In [9]:
print(name_list)
print(embedding_list)

['biplob', 'hritik', 'karan', 'rohan', '羅翎禎', '邱崑山']
[tensor([[ 0.0495, -0.0520, -0.0417, -0.0144, -0.0211,  0.0020,  0.0714, -0.0816,
          0.0025,  0.0548, -0.0169,  0.0676,  0.0562,  0.0160, -0.0331, -0.0229,
         -0.0720,  0.0145,  0.0372,  0.0727, -0.0121,  0.0021,  0.0013,  0.0072,
          0.0177,  0.0246, -0.0331, -0.0536, -0.0627, -0.1163,  0.0604,  0.0434,
         -0.0143,  0.0192,  0.0326, -0.0179,  0.0323,  0.0490, -0.0428, -0.0049,
         -0.0354,  0.0046, -0.0354, -0.0460, -0.0680, -0.0241, -0.0474, -0.0028,
         -0.0009, -0.0109, -0.0160, -0.0287, -0.0630,  0.0215, -0.0253, -0.0327,
         -0.0006,  0.0372, -0.0326,  0.0137,  0.0517,  0.0868,  0.0250, -0.0108,
          0.0422, -0.0215,  0.0577, -0.1190,  0.0288,  0.0567, -0.0245, -0.0192,
         -0.0076,  0.0661,  0.0178, -0.0178, -0.0033,  0.0137,  0.0433, -0.0203,
         -0.0284,  0.0341, -0.0312, -0.0622, -0.0217,  0.0146,  0.0531,  0.0108,
         -0.1267, -0.0181,  0.0224, -0.0176,  0.0135,  

In [10]:
test_transform=transforms.ToTensor()

In [11]:
webcam2.video_stream()   #開始從webcam串流影像
count = 0
Starttime=time.time()
HTML=""         #影片上方要顯示的文字 
while True:
  img = webcam2.video_frame(HTML)          #取得目前的影像frame，傳入對應的圖片說明
  if not img:
    break
  img = test_transform(img) #目前的影像作預處裡
  img=transforms.ToPILImage()(img)
  img_cropped_list, prob_list = mtcnn(img, return_prob=True)
  if img_cropped_list is not None:
    boxes, _ = mtcnn.detect(img)
    for i, prob in enumerate(prob_list):
      if prob>0.90:
        emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
        dist_list = [] # 儲存人臉特徵向量距離
        for idx, emb_db in enumerate(embedding_list):
          dist = torch.dist(emb, emb_db).item()
          dist_list.append(dist)
          min_dist = min(dist_list) # 得到最短距離，即最相似度
          min_dist_idx = dist_list.index(min_dist) #得到最短距離的索引
          Output = name_list[min_dist_idx] #得到最近的姓名
  count+=1
  Stoptime=time.time()
  HTML =str(round(count/(Stoptime-Starttime),2))+"FPS<br>"+Output

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored